In [1]:
import os
import mne
import pandas as pd
import numpy as np
import janitor
import warnings
from tqdm.notebook import tqdm
import dask.dataframe as dd

import torch
from torch.utils.data import Dataset

warnings.filterwarnings("ignore", category=RuntimeWarning)

data_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp'
final_dataset_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset'

/home/owner/Documents/DEV/BrainLabyrinth/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# # from sklearn.decomposition import PCA as sklearnPCA
# import scipy.signal as signal
# from mne.preprocessing import ICA


# for user_id in tqdm(os.listdir(data_path)):
#     person_dir = os.path.join(data_path, user_id)
#     for file in os.listdir(person_dir):
#         file_path = os.path.join(person_dir, file)

#         if file.endswith(".vhdr"):
#             # Read the EEG data
#             raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)
            
#             # Apply preprocessing steps

#             # 1. Resample to 500 Hz
#             raw.resample(500, npad="auto")  # Resample to 500 Hz
            
#             # 2. Filter high-pass at 0.1 Hz (for low-frequency noise removal)
#             raw.filter(l_freq=0.1, h_freq=30)  # High-pass filter (low-frequency cutoff at 0.1 Hz)
            
#             # 3. Notch filter at 50 Hz (to remove power line noise)
#             raw.notch_filter(freqs=50, picks='all')  # Notch filter for 50 Hz

#             # 4. ICA for blink removal
#             ica = ICA(n_components=20, method='fastica', max_iter=2000, random_state=97)
#             ica.fit(raw, picks='eeg')  # Fit to EEG channels excluding EOG
            
#             # Detect EOG artifacts
#             eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name='EOG', threshold=2.0)
#             ica.exclude = eog_indices
            
#             # Apply ICA cleaning
#             raw = ica.apply(raw)
#             raw.drop_channels(['EOG'])  # Remove auxiliary EOG channel after cleaning

#             # 4. Detrend the data using scipy.signal.detrend
#             raw._data = signal.detrend(raw._data, axis=1)  # Detrend along the time axis (axis=1)

#             # 5. Apply PCA for dimensionality reduction (if necessary)
#             # n_components = 20
#             # pca = sklearnPCA(n_components=n_components)
#             # data_pca = pca.fit_transform(raw.get_data())  # Apply PCA to the EEG data
#             # raw._data = data_pca  # Replace the original data with the PCA-transformed data
            
#             # Continue with the rest of your pipeline as you have it
#             # Find the corresponding .vmrk file
#             vmrk_file = file.replace('.vhdr', '.vmrk')
#             vmrk_file_path = os.path.join(person_dir, vmrk_file)

#             if os.path.exists(vmrk_file_path):
#                 # Read the annotations (markers) from the .vmrk file
#                 annotations = mne.annotations.read_annotations(vmrk_file_path)
#             # Convert raw data to DataFrame
#             time_series = raw.to_data_frame()

#             # Extract markers (annotations)
#             marker_times = annotations.onset  # In seconds
#             marker_labels = annotations.description  # The marker labels

#             # Create a DataFrame for the markers
#             markers_df = pd.DataFrame({
#                 'event_id': np.arange(len(marker_labels)), 
#                 'start': marker_times - 3.0,
#                 'end': marker_times + 1.0,
#                 'marker': marker_labels
#             })
            
#             markers_df = markers_df[
#                 ~markers_df.marker.isin([
#                     'Marker/Impedance', 'New Segment/', 'Stimulus/2'
#                 ])
#             ]
#             # markers_df.marker = markers_df.marker.replace({'Stimulus/2': 'Stimulus/P'})

#             # Display the first few rows of both DataFrames
#             # Merge markers with EEG data
#             time_series['time'] = time_series['time'].round(3)  # Round times to 3 decimal places for matching

#             merged_df = janitor.conditional_join(
#                 markers_df, 
#                 time_series,
#                 ('start', 'time', '<='),
#                 ('end', 'time', '>='),
#                 how='left',
#                 df_columns=['event_id', 'marker']
#             )
#             # display(merged_df.groupby('event_id').count())
#             merged_path = os.path.join(final_dataset_path, f"{user_id}.parquet")
#             merged_df.to_parquet(merged_path)


In [3]:
# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset copy'

# List all Parquet files in the directory
parquet_files = [os.path.join(parquet_directory, f) for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

# Initialize an empty list to hold the Dask DataFrames
dask_dfs = []

# Process each Parquet file
for file in parquet_files:
    # Read the Parquet file into a Dask DataFrame
    df = dd.read_parquet(file)

    # Extract the file name without the extension
    file_name = os.path.basename(file).replace('.parquet', '')

    # Create a unique event_id by combining the original event_id with the file name
    df['event_id'] = df['event_id'].astype(str) + '_' + file_name

    # Append the DataFrame to the list
    dask_dfs.append(df)

# Concatenate all DataFrames into a single Dask DataFrame
combined_df = dd.concat(dask_dfs, ignore_index=True)

# Create a mapping dictionary for unique event_id values
unique_event_ids = combined_df['event_id'].unique().compute()
event_id_mapping = {event_id: idx for idx, event_id in enumerate(unique_event_ids)}

# Renumber the event_id column using the mapping dictionary
combined_df['event_id'] = combined_df['event_id'].map(event_id_mapping, meta=('event_id', 'int64'))

# Optionally, you can perform operations on the combined Dask DataFrame
# For example, you can compute the first few rows to verify the data
print(combined_df.head())

# Save the combined Dask DataFrame to a new Parquet file
combined_df.to_parquet('/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet')


   event_id      marker   time        Fp1        Fpz       Fp2        F7  \
0         0  Stimulus/1  9.438  18.260990  16.979025  1.769803  2.441713   
1         0  Stimulus/1  9.440  18.279552  17.167071  1.736013  2.509046   
2         0  Stimulus/1  9.442  18.187657  17.195618  1.650717  2.619466   
3         0  Stimulus/1  9.444  17.988738  17.064795  1.536693  2.780392   
4         0  Stimulus/1  9.446  17.697904  16.795464  1.420282  2.984275   

         F3         Fz        F4  ...       PO3        PO4       PO6  \
0  2.818692  10.922015  5.462004  ... -4.601150 -10.072189  5.858149   
1  3.320807  11.627260  6.565130  ... -5.309328 -10.643972  4.269458   
2  3.746728  12.211854  7.602871  ... -5.970655 -11.330150  2.269126   
3  4.108368  12.665638  8.477080  ... -6.628905 -12.088001  0.015087   
4  4.421709  12.995026  9.112953  ... -7.331279 -12.876470 -2.320403   

        FT7       FT8        TP7       TP8       PO7       PO8         Oz  
0 -6.076038 -3.457559 -10.564211 -

In [ ]:
import dask.dataframe as dd

# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet'

# Read the Parquet files into a Dask DataFrame
dask_df = dd.read_parquet(parquet_directory)

# Compute the shape of the Dask DataFrame
rows, cols = dask_df.shape
rows_computed = rows.compute()

# Print the shape
print(f"Shape of the final dataset: ({rows_computed}, {cols})")

# Optionally, you can compute the first few rows to verify the data
print(dask_df.head())
dask_df.columns


Shape of the final dataset: (5828008, 66)
   event_id      marker   time        Fp1        Fpz       Fp2        F7  \
0         0  Stimulus/1  9.438  18.260990  16.979025  1.769803  2.441713   
1         0  Stimulus/1  9.440  18.279552  17.167071  1.736013  2.509046   
2         0  Stimulus/1  9.442  18.187657  17.195618  1.650717  2.619466   
3         0  Stimulus/1  9.444  17.988738  17.064795  1.536693  2.780392   
4         0  Stimulus/1  9.446  17.697904  16.795464  1.420282  2.984275   

         F3         Fz        F4  ...       PO3        PO4       PO6  \
0  2.818692  10.922015  5.462004  ... -4.601150 -10.072189  5.858149   
1  3.320807  11.627260  6.565130  ... -5.309328 -10.643972  4.269458   
2  3.746728  12.211854  7.602871  ... -5.970655 -11.330150  2.269126   
3  4.108368  12.665638  8.477080  ... -6.628905 -12.088001  0.015087   
4  4.421709  12.995026  9.112953  ... -7.331279 -12.876470 -2.320403   

        FT7       FT8        TP7       TP8       PO7       PO8      

Index(['event_id', 'marker', 'time', 'Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz',
       'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4',
       'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8',
       'POz', 'O1', 'O2', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6',
       'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1',
       'P2', 'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8',
       'PO7', 'PO8', 'Oz'],
      dtype='object')

In [ ]:
# import os
# ok = os.listdir('/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset (copy)')

# cleaned_dir = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset copy'
# for file in os.listdir(cleaned_dir):
#     if file not in ok:
#         os.remove(os.path.join(cleaned_dir, file))